# **Linear Algebra for Data Science**: HW3, task 5 (3 pts)
# Iterative methods for eigenvalue calculations

### <div align="right"> &copy; Yurii Yeliseev & Rostyslav Hryniv, 2023 </div>

## Completed by:   
*   Artur Shevtsov
*   Yurii Roziaiev

The aim of this task is to implement two iterative methods for eigenvalue and eigenvector calculations, the Jacobi and QR ones.


# 1. Jacobi method **(1 pt)**
The Jacobi method is applicable to symmetric matrices $A$ and uses the ***Givens rotations*** (also called  ***Jacobi rotations*** in this context) that consecutively decrease the off-diagonal part $$\mbox{off}(A):=\sum_{i\ne j} |a_{ij}|^2$$ and thus transform $A$ to "nearly" diagonal form. (See the corresponding task from the first homework for details on the Givens rotations.) If $$A_k = Q_kQ_{k-1}\cdots Q_2Q_1 A Q_1^\top Q_2^\top \cdots Q_{k-1}^\top Q_k^\top$$ is alsmost diagonal, then its diagonal entries are close to the eigenvalues of $A$, and the approximation to eigenvectors can be obtained from the Givens rotations.

## 1.1. Algorithm justification
#### 1.1.1. Basic step
Assume that $A$ is an $n\times n$ symmetric matrix and that indices $p,q$ with $1\le p < q \le n$ are such that $a_{pq} \ne 0$. The task is to find an angle $\theta$ such that, with $c:=\cos\theta$ and $s:=\sin\theta$, the matrix
$$\begin{pmatrix} b_{pp} & b_{pq} \\ b_{qp} & b_{qq} \end{pmatrix} = \begin{pmatrix} c & -s \\ s & c \end{pmatrix}\begin{pmatrix} a_{pp} & a_{pq} \\ a_{qp} & a_{qq} \end{pmatrix}\begin{pmatrix}c & s \\ -s & c  \end{pmatrix}$$ is diagonal.


#### **Task 1.1.2**: Explain why $|b_{pp}|^2 + |b_{qq}|^2 > |a_{pp}|^2 + |a_{qq}|^2$

____

We can explain it using the fact that Givens rotation does not change the Frobenius norm:

$$\| A \|_F^2 = \| B \|_F^2$$

$$\| A \|_F^2 = |a_{pp}|^2 + |a_{pq}|^2 + |a_{qp}|^2 + |a_{qq}|^2$$

$$\| B \|_F^2 = |b_{pp}|^2 + |b_{pq}|^2 + |b_{qp}|^2 + |b_{qq}|^2$$

Matrix $A$ is symmetric, so $a_{pq} = a_{qp} \implies \| A \|_F^2 = |a_{pp}|^2 + |a_{qq}|^2 + 2|a_{pq}|^2$.
Matrix $B$ is diagonal, so $b_{pq} = b_{qp} = 0 \implies \| B \|_F^2 = |b_{pp}|^2 + |b_{qq}|^2$.

Since $a_{pq} \neq 0 \implies 2|a_{pq}|^2 > 0$, therefore we prove, that:

$$|b_{pp}|^2 + |b_{qq}|^2 > |a_{pp}|^2 + |a_{qq}|^2$$

---


#### **Task 1.1.3**: Find the angle $\theta$ so that the above matrix $B$ is diagonal
---

For B to be diagonal, off-diagonal elements should be zero: $b_{pq} = b_{qp} = 0$.
If we multiply the matrices, we get the following expression for $b_{pq}:$

$$ b_{pq} = s(ca_{pp} - sa_{pq}) + c(ca_{pq} - sa_{qq})= a_{pq}(c^2 - s^2) + sc(a_{pp} - a_{qq}) = 0$$

Next, using trigonometric formulas for $\sin\theta\cos\theta = \frac{1}{2}\sin2\theta$ and $\cos^2\theta - \sin^2\theta = \cos2\theta$:

$$a_{pq}\cos2\theta + \frac{1}{2}\sin2\theta(a_{pp} - a_{qq}) = 0$$

Solving for $\theta$ we get:

$$\tan2\theta = \frac{2a_{pq}}{a_{qq} - a_{pp}}$$

$$\theta = \frac{1}{2}\arctan\frac{2a_{pq}}{a_{qq} - a_{pp}}$$

If $a_{pp} = a_{qq} \implies \theta = \frac{\pi}{4}$
If $a_{pq} = 0 \implies \theta = 0 \implies$ matrix is diagonal

---  


### **Task 1.1.4**: write a function that for a given matrix $A$ and indices $p<q$ finds the angle $\theta$ in the Jacobi rotation $J$  


In [8]:
import numpy as np
def calc_rotation_angle(A, p, q):
    return 0.5 * np.arctan2(2 * A[p, q], A[q, q] - A[p, p])

## 1.2 The algorithm and its implementation
Denote by $$J(p,q,\theta): = \begin{pmatrix}1 & \dots & 0 & \dots & 0 &  \dots & 0 \\ \vdots & \ddots & \vdots & & \vdots & & \vdots \\ 0 & \dots & c & \dots & s & \dots & 0 \\ \vdots & \ddots & \vdots & & \vdots & & \vdots \\ 0 & \dots & -s & \dots & c & \dots & 0 \\ \vdots & \ddots & \vdots & & \vdots & & \vdots \\ 0 & \dots & 0 & \dots & 0 &  \dots & 1 \end{pmatrix}$$ the Jacobi (Givens) rotation in entries $p<q$.

Given a symmetric $n\times n$ matrix $A$ and a positive
tolerance $\mbox{tol}$, we find an orthogonal $V = J_1 J_2\cdots J_k$ such that $B:= V^\top A V$ has small off-diagonal part, namely $\mbox{off}(B) \le \delta:= \mbox{tol}\cdot \|A\|_F$.

#### **Initialize:** $V = I_n$, $B=A$
#### **while** $\mbox{off}(B) > \delta$:  
- find $p<q$ s.t. $|b_{pq}| = \max_{i\ne j}|b_{ij}|$
- set $\theta = \texttt{rot_angle}(B, p, q)$
- update $B = J(p,q,\theta)^\top B J(p,q,\theta)$
- update $V = V J(p,q,\theta)$

#### **end**




---
### **Task 1.2.1**: Implement the above algorithm

In [9]:
def jacobi_method(A, tol=1e-8, max_iter=1000):
    """
    The jacobi_method function is used to calculate the eigenvalues and eigenvectors of a square matrix using the Jacobi method.

    Inputs:

    A (numpy.ndarray): The square matrix for which the eigenvalues and eigenvectors are to be calculated.
    tol (float): The tolerance level. The default value is 1e-8.
    max_iter (int): The maximum number of iterations to perform. The default value is 1000.
    Outputs:

    numpy.ndarray: The eigenvalues of the input matrix.
    numpy.ndarray: The eigenvectors of the input matrix.
    """
    n = A.shape[0]
    V = np.identity(n)
    B = A.copy()

    off_B = np.linalg.norm(np.triu(B, k=1), 'fro')  
    delta = tol * off_B
    
    i = 0
    while off_B > delta and i < max_iter:
        # Find index of pivot element b_pq and angle theta
        p, q = np.unravel_index(np.argmax(np.abs(np.triu(B, k=1))), B.shape) 
        theta = calc_rotation_angle(B, p, q)
        
        # Construct rotation matrix
        c, s = np.cos(theta), np.sin(theta)
        J = np.identity(n)
        J[p, p], J[p, q] =  c, s
        J[q, p], J[q, q] = -s, c
        
        # Rotate B and update total rotation matrix V
        B = J.T @ B @ J
        V = V @ J
        
        # Update off-diagonal Frobenius norm
        off_B = np.linalg.norm(np.triu(B, k=1), 'fro') 
        i += 1
    
    eigenvalues, eigenvectors = np.diag(B), V
    return eigenvalues, eigenvectors

Observe that on each step the norm of the off-diagonal part of $B$ decreases at least by the factor $(N-1)/N$, where $N:=n(n-1)$. Therefore, the algorithm converges in a finite number of steps

---
### **Task 1.2.2**: Eigenvalues and eigenvectors by Jacobi method

##### Given that $B = V^\top AV$ is almost diagonal, identify the approximate eigenvalues and eigenvectors of $A$
---

This identification comes from the fact that the transformation $B = V^\top AV$ diagonalizes $A$, and the eigenvectors are obtained as columns of $V$ while the eigenvalues appear on the diagonal of $B$. This is a fundamental property of diagonalization, and it allows us to express $A$ in terms of its eigenvalues and eigenvectors.

---

# 2. QR-method **(1 pt)**

The $QR$ eigenvalue method is based on the observation that, under mild assumptions on an $n\times n$ matrix $A$, the sequence constructed starting with $A_1 = A$ and the update step $$A_k = Q_kR_k \quad \mapsto \quad A_{k+1}:=R_kQ_k$$
coverges to an upper-triangular $A_*$ holding the eigenvalues of the initial matrix $A$


## 2.1 Justification of the method

The fact that the sequence $A_k$ converges to an upper-triangular limit is rather difficult to prove and is skipped here. We will justify that the method gives the eigenvalues of $A$ and, in symmetric case, also the corresponding eigenvectors

### 2.1.1. Eigenvalues
Explain why the diagonal entries of the upper-triangular limiting matrix $A_* = \lim_{k\to\infty} A_k$ are eigenvalues of $A$ *(Hint: show that all matrices $A_k$ are similar to $A$)*

---
Two n-by-n matrices A and B are called similar if there exists an invertible n-by-n matrix P such that $ B=P^{-1}AP $. 
Hence, we can transform our sequence as follows:

$$ A_{k+1} = R_{k}Q_{k} = Q_{k}^{-1}Q_{k}R_{k}Q_{k} = Q_{k}^{-1}A_{k}Q_{k} = Q_{k}^\top A_{k}Q_{k}, $$

where $Q$ is orthogonal and we can clearly see that $A_k$ and $A_{k+1}$ are similar, therefore they have the same eigenvalues. 

---

### 2.1.2. Eigenvectors
Assume now that the initial matrix $A$ is symmetric. Explain why all matrices $A_k$ are also symmetric. Conclude that the limiting matrix $A_*$ is diagonal. Then identify an orthogonal matrix $V$ such that $A_* = V^\top AV$ and thus find the corresponding eigenvectors of $A$

---

You're correct, and I appreciate your attention to detail. Let's provide a more explicit proof for the symmetry of \(A_k\) in the case where the initial matrix \(A\) is symmetric.

Given that $A$ is symmetric, i.e., $A^\top = A$, let's show that $A_k$ is symmetric:

$$ A_{k+1} = Q_k^\top A_k Q_k $$

Now, taking the transpose of both sides:

$$ A_{k+1}^\top = (Q_k^\top A_k Q_k)^\top $$

Using the property $(AB)^\top = B^\top A^\top$:

$$ A_{k+1}^\top = Q_k^\top A_k^\top (Q_k^\top)^\top $$

Since $A_k$ is symmetric $A_k^\top = A_k$:

$$ A_{k+1}^\top = Q_k^\top A_k Q_k \implies A_{k+1} \text{ is also symmetric}$$

Matrix $Q$ is orthogonal, so the transformation above diagonalizes $A_k$, so that the limiting matrix $A_*$ is diagonal and is made of eigenvalues.
After going through all required $k$ iterations, the expression for $A_*$ looks like the following:

$$ A_* = (Q^\top_kQ^\top_{k-1} \cdots Q^\top_2 Q^\top_1)A(Q_1 Q_2 \cdots Q_{k-1}Q_k) = V^\top AV,$$

where $V$ is the product of $k$ orthogonal matrices $Q_i$: $V = Q_1 Q_2 \cdots Q_k$

---

## 2.2. Implementation of the method
No need to implement the QR factorization from the scratch (although you may use your solutions of the HW1 based on the Givens rotations and/or Householder reflections)

In [10]:
def QR_method(A, tol=1e-10, max_iter=1000):
    """The QR_method function is used to calculate the eigenvalues of a square matrix using the QR method.

    Inputs:

    A (numpy.ndarray): The square matrix for which the eigenvalues are to be calculated.
    tol (float): The tolerance level. The default value is 1e-10.
    max_iter (int): The maximum number of iterations to perform. The default value is 1000.
    Outputs:

    numpy.ndarray: The diagonal of the final transformed matrix A, which represents the eigenvalues of the input matrix"""
    V = np.identity(A.shape[0])
   
    off_A = np.linalg.norm(np.triu(A, k=1), 'fro')  
    delta = tol * off_A
    
    i = 0
    while off_A > delta and i < max_iter:
        Q, R = np.linalg.qr(A)
        A = R @ Q
        V = V @ Q
        
        off_A = np.linalg.norm(np.triu(A, k=1), 'fro')
        i += 1

    eigenvalues, eigenvectors = np.diag(A), V
    return eigenvalues, eigenvectors, A

# 3. Testing **(0.5 pts)**

## 3.1 Initializing $A$
Generate a $10\times 10$ matrix $A = QDQ^{\top}$, where $Q$ is a (randomly generated) orthogonal matrix  and $D=\mbox{diag}\{1,2,\dots,10\}$ is a diagonal matrix. To generate $Q$, start with a $10\times 10$ matrix $B$ with random entries in $(0,1)$ and apply the $QR$-factorisation of $B$, $B=QR$, to determine $Q$. Alternatively, apply Gram-Schmidt to find an orthonormal basis of $\mathbb{R}^{10}$

In [11]:
np.random.seed(42)
B = np.random.rand(10, 10)
Q, _ = np.linalg.qr(B)
D = np.diag(np.arange(1, 11))
A = Q @ D @ Q.T

## 3.2 Eigenvalue and eigenvector calculations

### 3.2.1. Jacobi method
Apply Jacobi method to determine the eigenvalues and eigenvectors

In [12]:
eigenvalues_jac, eigenvectors_jac = jacobi_method(A)

### 3.2.2 QR method
Apply the QR method to determine the eigenvalues and eigenvectors

In [13]:
eigenvalues_qr, eigenvectors_qr, arr = QR_method(A)

## 3.3 Evaluating the results

Compare the results with the built-in function and comment on the outcomes

In [14]:
eigenvalues_eig, eigenvectors_eig = np.linalg.eig(A)
print(f"\nEigenvalues (Jacobi):\n{np.sort(eigenvalues_jac, axis=0)}")
print(f"\nEigenvectors (Jacobi):\n{np.take_along_axis(eigenvectors_jac, np.argsort(np.abs(eigenvectors_jac), axis=1), axis=1)}")
print(f"\nEigenvalues (QR):\n{np.sort(eigenvalues_qr, axis=0)}")
print(f"\nEigenvectors (QR):\n{np.take_along_axis(eigenvectors_qr, np.argsort(np.abs(eigenvectors_qr), axis=1), axis=1)}")
print(f"\nEigenvalues (np.linalg.eig):\n{np.sort(eigenvalues_eig, axis=0)}")
print(f"\nEigenvectors (np.linalg.eig):\n{np.take_along_axis(eigenvectors_eig, np.argsort(np.abs(eigenvectors_eig), axis=1), axis=1)}")



Eigenvalues (Jacobi):
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]

Eigenvectors (Jacobi):
[[ 0.01672616 -0.08449834 -0.09898519 -0.09898765  0.20476003 -0.31899286
  -0.35938116 -0.45810392  0.46791825  0.52090212]
 [-0.01034554  0.01125349  0.03456666 -0.11105043  0.15380457  0.1596036
   0.22421966 -0.2361641   0.64104822  0.64816616]
 [ 0.02335539 -0.0514379   0.07255162  0.15405075 -0.18348828 -0.283905
  -0.32697968  0.33449827 -0.55105251 -0.57539519]
 [-0.14671391 -0.16073056  0.23265842 -0.25077656  0.31036189  0.33214307
  -0.37393719  0.38489824  0.39423473 -0.43079191]
 [ 0.06671796 -0.08403261  0.13726203 -0.15486168  0.26121054  0.27678381
  -0.33520468 -0.45811054 -0.48787294 -0.49048604]
 [-0.0518838   0.05412371  0.08076804  0.13098211 -0.18208943  0.26614203
  -0.31095076 -0.37658907  0.5300692   0.5892603 ]
 [ 0.0059725  -0.06644272 -0.13068058  0.173022   -0.18400638  0.21248879
   0.29050153 -0.31741494  0.4650144   0.68421053]
 [ 0.07176493  0.15008553  0.172725   -

---

As we can see, both methods correctly computed the eigenvalues and eigenvectors when comparing to the built-in NumPy function. 

---


# 4. Conclusions **(0.5 pts)**
Summarize in several sentences what you achieved in this task, what were the main obstacles and how you overcome them. You can also add some suggestions how this task can be improved in the future

---

In this task we implemented two iterative methods for eigenvalue and eigenvector calculations, the Jacobi and QR ones. In general, both methods orthogonally diagonalize the initial symmetric matrix $A$, but use different orthogonal matrices for it. One of the main obstacles of this task was the algorithm justification part, but after reading some additional literature, we were able to overcome it. 

---